# **Redis Lab: Querying the Movies Dataset**

## **Objective**
This lab will help you practice querying data in Redis using the dataset provided [here](https://redis.io/learn/howtos/moviesdatabase/import#importing-the-movies-theaters-and-users). You will execute a series of progressively challenging queries to retrieve, filter, and manipulate movie-related data.

## **Dataset Overview**
The dataset consists of the following key types:
- **Movies**: Stored as hashes with keys like `movie:<id>`
- **Theaters**: Stored as hashes with keys like `theater:<id>`
- **Users**: Stored as hashes with keys like `user:<id>`
- **Ratings**: Stored as sorted sets to track movie ratings
- **Indexes**: Secondary indexes to facilitate searching

---

## **Lab Instructions**
**Perform the following queries in Redis CLI.**

In [21]:
import redis

r = redis.Redis(decode_responses=True)

### **1. Retrieve a Movie by ID**
Fetch details of the movie with ID `1`.

In [ ]:
movie = r.hget(name='movie:1', key='title')
print(movie)

Guardians of the Galaxy


### **2. List All Movies**
Retrieve all movie IDs stored in Redis.

In [ ]:
ids = r.keys("movie:*")


922

In [118]:
# scan iter for more optimal approach
ids2 = list(r.scan_iter(match="movie:*"))
    

### **3. Find Movies by Title Prefix**
Find all movies that start with "The Lord of the Rings".

In [ ]:
results = []

for key in r.scan_iter(match="movie:*"):
    title = r.hget(key, "title")
    if "The Lord of the Rings" in title:
        results.append(title)

### **4. Retrieve Theaters in a Specific City**
Find all theaters located in "New York".

In [47]:
nyc_theaters = []

for key in r.scan_iter(match="theater:*"):
    if r.hget(name=key, key="city") == "New York":
        nyc_theaters.append(r.hget(name=key, key="name"))
print(nyc_theaters)

['The Walter Reade Theater at Lincoln Center', 'P.S.122 Performance Space', "Lamb's Theatre", 'Rattlestick Theater', 'Minskoff Theatre', 'Booth Theatre', 'The Little Shubert theatre', 'Vineyard Theatre', 'Al Hirschfeld Theater', 'Manhattan Theatre Source', 'Theatre 80 St Marks', 'SoHo Playhouse', 'Palace Theatre', 'New Victory Theatre', 'Belasco Theatre', 'Marquis Theatre', 'The Cherry Pit', 'St. James Theatre', 'Union Square Theater', 'Majestic Theatre', 'Acorn Theater', 'August Wilson Theatre', 'Henry Miller Theatre', 'Arclight Theatre', 'New World Stages', '45th Street Theater', 'Mazer Theatre', 'Lucille Lortel Theatre', "Eugene O'Neill Theater", 'Neil Simon Theatre', '59E59', 'Theatre Row', 'Ontological Hysteric Theater', 'Barrow Street Theatre', 'Kraine Theater', 'Snapple Theater Center', 'Pearl Theatre', 'Studio 54 Theatre', 'Shubert Theatre', 'Next Stage Theater', 'Laura Pels Theater', 'Orpheum Theater', 'Village Theater', 'Harold Clurman Theater', 'Nokia Theatre Times Square', 

### **5. Get Top-Rated Movies**
Retrieve the top 5 movies based on rating.

In [92]:
movies = [None] * 5
ratings = [0] * 5

for key in r.scan_iter(match="movie:*"):
    rating = float(r.hget(name=key, key="rating"))
    if rating > min(ratings):
        # drop the last movie and rating in the list
        movies[4] = [key, r.hget(name=key, key="title")]
        ratings[4] = rating
        # sort based on new ones
        sorted_pairs = sorted(zip(ratings, movies), reverse=True)
        ratings, movies = zip(*sorted_pairs)
        ratings, movies = list(ratings), list(movies)

print(movies)
print(ratings)


[['movie:787', 'Vegas (doc)'], ['movie:521', 'Boy 9'], ['movie:314', 'The Shawshank Redemption'], ['movie:723', 'Ween Live in Chicago'], ['movie:1025', 'Over Canada: An Aerial Adventure']]
[9.4, 9.4, 9.3, 9.2, 9.1]


### **6. Find Movies Released After 2010**
Retrieve movies where the release year is greater than 2010.

In [100]:
movies_after_2010 = []

for key in r.scan_iter(match="movie:*"):
    name = r.hget(name=key, key="title")
    year = int(r.hget(name=key, key="release_year"))
    if year > 2010:
        movies_after_2010.append(name)
print(movies_after_2010)

['The Boat', 'Iron Man 3', 'Star Wars: Episode IX - The Rise of Skywalker', 'Teenage Mutant Ninja Turtles: Out of the Shadows', 'Captain Fantastic', 'Non-Stop', 'The Best of Me', 'Midnight,Texas', "The Village: Achiara's Secret", 'Teenage Mutant Ninja Turtles', 'Occupy Los Angeles', 'Mascots', 'Barcelona: A Love Untold', 'Sully', 'En man som heter Ove', 'Gone Girl', 'Snowden', 'Chicago Fire', 'Miami Monkey', 'Spider-Man: Far from Home', 'Southpaw', 'Monster High: Boo York,Boo York', 'The Neon Demon', 'Undercover Boss Canada', 'Hunt for the Wilderpeople', 'Kingsman: The Secret Service', 'Deidra & Laney Rob a Train', 'The Village 1', 'The Man Who Knew Infinity', 'What We Do in the Shadows', 'Furious Seven', 'Boy 7', "Miss Peregrine's Home for Peculiar Children", 'San Andreas', 'Captain America: The Winter Soldier', 'The Amazing Spider-Man 2', 'Bleed for This', 'Ghostbusters', '7 Days to Vegas', 'The Nice Guys', 'Ouija: Origin of Evil', 'California (2018)', 'Transcendence', 'Anthropoid', 


### **7. Get Movies by Genre**
Find all movies labeled as "Action".

In [106]:
action_movies = []

for key in r.scan_iter(match="movie:*"):
    name = r.hget(name=key, key="title")
    genre = r.hget(name=key, key="genre")
    if genre == "Action":
        action_movies.append(name)
print(action_movies)

['Iron Man 3', 'Star Wars: Episode IX - The Rise of Skywalker', 'Teenage Mutant Ninja Turtles: Out of the Shadows', 'Non-Stop', 'The Running Man', 'Teenage Mutant Ninja Turtles', 'Texas Rangers', 'Chicago Fire', 'Spider-Man: Far from Home', 'Battle in Seattle', 'The Last of the Mohicans', 'Kingsman: The Secret Service', 'Furious Seven', 'San Andreas', 'Rome: The Other Side of Violence', 'Captain America: The Winter Soldier', 'The Amazing Spider-Man 2', 'Ghostbusters', 'The Nice Guys', 'Harley Davidson and the Marlboro Man', 'The Siege of Jadotville', 'Star Wars: Episode I - The Phantom Menace', 'In the Heart of the Sea', 'Money Train', 'American Sniper', 'The Fifth Element', 'Deadpool', 'Gods of Egypt', 'California Cowboys', 'Point Break', 'A Bullet for Pretty Boy', 'Blood Father', 'Aftershock: Earthquake in New York', 'NXT TakeOver: Chicago II', 'Spider-Man 2', 'Chicago Overcoat', 'Jurassic World', 'Fury', 'Walker,Texas Ranger', 'Star Wars: Episode IV - A New Hope', 'Mr. Right', 'Sica

### **8. Find Users Who Rated a Specific Movie**
Retrieve all users who have rated the movie with ID `10`.


In [109]:
# not possible

### **9. Get Movies with Ratings Between 7 and 9**
Find all movies that have a rating between 7.0 and 9.0.

In [ ]:
[r.hget(name=key, key="title")
for key in r.scan_iter(match="movie:*")
if 7 < float(r.hget(name=key, key="rating")) < 9]

## **Additional Challenges**
- Modify query #5 to return the top 10 rated movies.

In [114]:
movies = [None] * 10
ratings = [0] * 10

for key in r.scan_iter(match="movie:*"):
    rating = float(r.hget(name=key, key="rating"))
    if rating > min(ratings):
        # drop the last movie and rating in the list
        movies[9] = [key, r.hget(name=key, key="title")]
        ratings[9] = rating
        # sort based on new ones
        sorted_pairs = sorted(zip(ratings, movies), reverse=True)
        ratings, movies = zip(*sorted_pairs)
        ratings, movies = list(ratings), list(movies)

print(movies)
print(ratings)


[['movie:787', 'Vegas (doc)'], ['movie:521', 'Boy 9'], ['movie:314', 'The Shawshank Redemption'], ['movie:723', 'Ween Live in Chicago'], ['movie:1025', 'Over Canada: An Aerial Adventure'], ['movie:664', 'PNYC: Portishead - Roseland New York'], ['movie:870', 'Bruce Springsteen & the E Street Band: Live in Barcelona'], ['movie:719', 'Jeff Buckley: Live in Chicago'], ['movie:1135', 'Pulp Fiction'], ['movie:1031', 'Canada på tvers med Lars Monsen']]
[9.4, 9.4, 9.3, 9.2, 9.1, 9.0, 8.9, 8.9, 8.9, 8.9]


- Find the 3 most recent movies added to the dataset.

In [117]:
# not possible


- Retrieve the total number of users who have rated at least one movie.

In [116]:
# not possible